In [29]:
import numpy as np
import pandas as pd

import time

import warnings

In [31]:
data = pd.read_csv('output (1).csv', index_col=False)

In [32]:
print(data.shape)

(1, 47)


In [33]:
array=data.values

In [34]:
array

array([[100, 0, 0.0, 2412192200, 61000000, 13244, 192200, 2412, 2306,
        911, 5969, 810, 13992, 1695, 1122033441, 8999840, 89050840,
        'NETFLIXUS', 0.0, 'T', 840, 840, '102510800600084063368',
        '01001324424121922000000001399200000001695', 'ECOM',
        'Kg1WR6lwTruEPIDK0GS4w82/wrFeXTU5SjD9TfyUXmc=', 'MASTER', False,
        1020001031, 0, 0, False, True, False, False, 'ECOM', 'ECOM',
        False,
        '2a0d5647a9704ebb843c80cad064b8c3cdbfc2b15e5d35394a30fd4dfc945e67',
        False, 'EKCZSH8MA5', False, False, False, True, 28.6448,
        77.216721]], dtype=object)

In [35]:
new_data=data.drop(["mti","processingCode","cardholderBillingConversionRate","stan","expiryDate","conversionDate","posEntryMode","forwardingInstitutionCode","rrn","additionalData48","transactionCurrencyCode",
"cardholderBillingCurrencyCode", "posDataCode","originalDataElement","encryptedPan","network","dcc","kitNo","factorOfAuthorization","authenticationScore","contactless","international",
"preValidated","enhancedLimitWhiteListing","encryptedHexCardNo","isTokenized","moneySendTxn","mcRefundTxn","mpqrtxn","authorisationStatus","cardAcceptorTerminalId", "cardAcceptorNameLocation","isExternalAuth"], axis=1)


In [41]:
new_data.head(5)


,transactionAmount,dateTimeTransaction,timeLocalTransaction,dateLocalTransaction,merchantCategoryCode,acquiringInstitutionCode,cardAcceptorId,cardBalance,channel,transactionOrigin,transactionType,entityId,latitude,longitude
0,0.0,2412192200,192200,2412,5969,13992,89050840,0.0,ECOM,ECOM,ECOM,EKCZSH8MA5,28.6448,77.216721


In [42]:
new_data.shape


(1, 14)

In [45]:
import pandas as pd

def check_rule_001(transactions_df):
    # Selecting relevant columns for the condition
    relevant_columns = [
        'transactionAmount', 'dateTimeTransaction', 'timeLocalTransaction',
        'dateLocalTransaction', 'merchantCategoryCode', 'acquiringInstitutionCode',
        'cardAcceptorId', 'cardBalance', 'channel', 'transactionOrigin',
        'transactionType', 'entityId', 'latitude', 'longitude'
    ]
    
    # Filter dataframe to include only relevant columns
    relevant_transactions = transactions_df[relevant_columns]

    # Convert transactionAmount and cardBalance to numeric
    relevant_transactions['transactionAmount'] = pd.to_numeric(relevant_transactions['transactionAmount'], errors='coerce')
    relevant_transactions['cardBalance'] = pd.to_numeric(relevant_transactions['cardBalance'], errors='coerce')

    # Convert dateTimeTransaction to timestamp
    relevant_transactions['dateTimeTransaction'] = pd.to_datetime(relevant_transactions['dateTimeTransaction'], errors='coerce')

    # Filter transactions within the last 12 hours
    twelve_hours_ago = pd.Timestamp.now() - pd.Timedelta(hours=12)
    recent_transactions = relevant_transactions[relevant_transactions['dateTimeTransaction'] >= twelve_hours_ago]

    # Group by entityId and calculate cumulative transaction amount for each entity
    cumulative_amount_per_entity = recent_transactions.groupby('entityId')['transactionAmount'].sum()

    # Calculate the maximum card balance for each entity
    max_card_balance_per_entity = relevant_transactions.groupby('entityId')['cardBalance'].max()

    # Check if the total cumulative amount >= 70% of the card balance and balance >= Rs 3,00,000 for any entity
    for entity_id, cumulative_amount in cumulative_amount_per_entity.items():
        max_balance = max_card_balance_per_entity.get(entity_id, 0)
        if cumulative_amount >= 0.7 * max_balance and max_balance >= 300000:
            return True

    return False



In [46]:
check_rule_001(new_data)

False

In [48]:
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles.
    return r * c

# Example usage:
# distance = haversine(lat1, lon1, lat2, lon2)

def check_rule_002(transactions_df, entity_id):
    # Selecting relevant columns for the condition
    relevant_columns = [
        'transactionAmount', 'dateTimeTransaction', 'timeLocalTransaction',
        'dateLocalTransaction', 'merchantCategoryCode', 'acquiringInstitutionCode',
        'cardAcceptorId', 'cardBalance', 'channel', 'transactionOrigin',
        'transactionType', 'entityId', 'latitude', 'longitude'
    ]
    
    # Filter dataframe to include only transactions for the specified entityId
    entity_transactions = transactions_df[transactions_df['entityId'] == entity_id][relevant_columns]

    # Convert transactionAmount and cardBalance to numeric
    entity_transactions['transactionAmount'] = pd.to_numeric(entity_transactions['transactionAmount'], errors='coerce')
    entity_transactions['cardBalance'] = pd.to_numeric(entity_transactions['cardBalance'], errors='coerce')

    # Convert dateTimeTransaction to timestamp
    entity_transactions['dateTimeTransaction'] = pd.to_datetime(entity_transactions['dateTimeTransaction'], errors='coerce')

    # Filter transactions within the last 12 hours
    twelve_hours_ago = pd.Timestamp.now() - pd.Timedelta(hours=12)
    recent_transactions = entity_transactions[entity_transactions['dateTimeTransaction'] >= twelve_hours_ago]

    # Check if total transaction amount within 12 hours is more than Rs 1,00,000
    total_transaction_amount = recent_transactions['transactionAmount'].sum()
    if total_transaction_amount < 100000:
        return False

    # Group transactions by location and count unique locations
    unique_locations = recent_transactions.groupby(['latitude', 'longitude']).size().reset_index(name='count')

    # Check if the user transacts from more than 5 locations with a minimum difference of 200KM between two locations
    if len(unique_locations) <= 5:
        return False

    # Check the distance between each pair of locations
    for i in range(len(unique_locations)):
        for j in range(i + 1, len(unique_locations)):
            lat1, lon1 = unique_locations.iloc[i]['latitude'], unique_locations.iloc[i]['longitude']
            lat2, lon2 = unique_locations.iloc[j]['latitude'], unique_locations.iloc[j]['longitude']
            distance = calculate_distance(lat1, lon1, lat2, lon2)
            if distance < 200:
                return False

    return True




In [50]:
check_rule_002(new_data,new_data['entityId'])

False